In [9]:
import re
import cbor2
import datetime
import pandas
import altair

from os import scandir

altair.data_transformers.disable_max_rows()

def custom_theme():
    return {
        'config': {
            "background": "#333",
            "title": {"color": "#fff"},
            "style": {"guide-label": {"fill": "#fff"}, "guide-title": {"fill": "#fff"}},
            "axis": {"domainColor": "#fff", "gridColor": "#888", "tickColor": "#fff"},
            'view': {
                'height': 750,
                'width': 1000,
            },
            'range': {
                #'category': {'scheme': 'bluepurple'}
            }
        }
    }

altair.themes.register('custom_theme', custom_theme)

# enable the newly registered theme
altair.themes.enable('custom_theme')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [43]:
filedata = list()
measurement = set()
for direntry in scandir("../target/criterion/data/main/blocksize/"):
    if direntry.is_dir():
        algo = direntry.name
        for blocksizeentry in scandir(direntry.path):
            if blocksizeentry.is_dir():
                blocksize = int(blocksizeentry.name)
                local_measurement = max(dataentry.name for dataentry in scandir(blocksizeentry.path) if blocksizeentry.is_file)
                filedata.append((algo, blocksizeentry.name, local_measurement))
 
data = list()
for algo, block, filename in filedata:
    with open(f"../target/criterion/data/main/blocksize/{algo}/{block}/{filename}", "rb") as file:
        obj = cbor2.load(file)
        for value in obj["values"]:
            data.append((algo, int(block), value))

raw_df = pandas.DataFrame(data, columns=["algorithm", "blocksize", "time"])
raw_df["time"] /= 1_000_000

In [44]:
#raw_df = raw_df[raw_df["algorithm"] != "in_place_fastx_parallel"];

points = altair.Chart(raw_df).mark_line(point=True).encode(
    x=altair.X('blocksize', scale=altair.Scale(type="log", base=2)),
    y=altair.Y('median(time)'),
    color='algorithm'
)

error = altair.Chart(raw_df).mark_errorband(extent='iqr').encode(
    x=altair.X('blocksize:Q', scale=altair.Scale(type="log", base=2)),
    y=altair.Y('time:Q'),
    color='algorithm',
)


points + error

alt.LayerChart(...)